# melon 차트 데이터 가져오기

In [1]:
#!pip install bs4

In [2]:
#!pip install selenium

In [1]:
import pandas as pd

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep
import requests
import json
import re

In [3]:
# sqlite3 모듈 사용

import sqlite3

In [4]:
connection = sqlite3.connect('melon_chart_data.db')
connection.row_factory = sqlite3.Row

cursor = connection.cursor()

In [5]:
sql = '''
CREATE TABLE melonchart (
    song_id INTEGER,
    year INTEGER,
    month TEXT,
    week TEXT,

    rank INTEGER
)
'''
cursor.execute(sql)

In [6]:
cursor.execute('SELECT * FROM melonchart')

rows = cursor.fetchall()


In [7]:
header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'
}


d  = webdriver.Chrome('chromedriver.exe')
d.implicitly_wait(3)

In [8]:
driver = d

def get_song_ids_and_ranks(bottom):
    song_ids = driver.find_elements_by_xpath(f'//*[@id="lst{bottom}"]/td[4]/div/a')
    song_ids = [re.sub('[^0-9]', '', song_id.get_attribute("href")) for song_id in song_ids]
    ranks = driver.find_elements_by_xpath(f'//*[@id="lst{bottom}"]/td[2]/div/span[1]')
    return song_ids, ranks

In [9]:
def insert_song_data(song_data):
    fields = ', '.join(song_data.keys())
    cursor.execute(f'INSERT INTO melonchart ({fields}) VALUES (?, ?, ?, ?, ?)', list(song_data.values()))
    connection.commit()
    

In [11]:
# melon 접속
d.get('http://www.melon.com/chart/index.htm')
d.get("http://www.melon.com/chart/search/index.htm")

d.find_element_by_xpath('//*[@id="d_chart_search"]/div/h4[1]/a').click()

age_xpath = '//*[@id="d_chart_search"]/div/div/div[1]/div[1]/ul/li[1]/span/label'
age = d.find_element_by_xpath(age_xpath)
age.click()

result = list()
for i in range(1, 11):
    year_xpath = '//*[@id="d_chart_search"]/div/div/div[2]/div[1]/ul/li[' + str(i) + ']/span/label'
    year = d.find_element_by_xpath(year_xpath)
    year.click()

    for i in range(1, 13):    
        month_xpath = '//*[@id="d_chart_search"]/div/div/div[3]/div[1]/ul/li[' + str(i) + ']/span/label'
        month = d.find_element_by_xpath(month_xpath)
        month.click()

        for i in range(1, 6):
            try:
                week_xpath = '//*[@id="d_chart_search"]/div/div/div[4]/div[1]/ul/li[' + str(i) + ']/span/label'
                week = d.find_element_by_xpath(week_xpath)
                week.click()

            except:
                print("week_xpath not found")
                continue


            classCd = d.find_element_by_xpath('//label[@for = "gnr_1"]')
            if '종합' not in classCd.text:
                classCd = d.find_element_by_xpath('//label[@for = "gnr_2"]')

            classCd.click()

            d.find_element_by_xpath('//*[@id="d_srch_form"]/div[2]/button/span/span').click() #검색 클릭
            sleep(5) 
            count = 0

            while count < 2: 
                song_ids, ranks = get_song_ids_and_ranks(count == 0 and 50 or 100)

                for rank, song_id in zip(ranks, song_ids):
                    sleep(5)

                    song_data = {
                        'song_id': song_id,
                        'year': re.sub('[^0-9]', '', year.text),
                        'month': month.text,
                        'week': week.text,
                        'rank': rank.text,
                    }

                    print(song_data)

                    result.append(song_data)
                    insert_song_data(song_data)

                d.find_element_by_xpath('//*[@id="frm"]/div[2]/span/a').click()
                count += 1


{'song_id': '31477685', 'year': '2019', 'month': '01월', 'week': '12.31~01.06', 'rank': '1'}
{'song_id': '31532643', 'year': '2019', 'month': '01월', 'week': '12.31~01.06', 'rank': '2'}
{'song_id': '31406357', 'year': '2019', 'month': '01월', 'week': '12.31~01.06', 'rank': '3'}
{'song_id': '31417871', 'year': '2019', 'month': '01월', 'week': '12.31~01.06', 'rank': '4'}
{'song_id': '31448480', 'year': '2019', 'month': '01월', 'week': '12.31~01.06', 'rank': '5'}
{'song_id': '31506637', 'year': '2019', 'month': '01월', 'week': '12.31~01.06', 'rank': '6'}
{'song_id': '31532438', 'year': '2019', 'month': '01월', 'week': '12.31~01.06', 'rank': '7'}


WebDriverException: Message: chrome not reachable
  (Session info: chrome=78.0.3904.108)


In [12]:
cursor.execute('SELECT * FROM melonchart')

rows = cursor.fetchall()
for row in rows:
    print()
    for key in row.keys():
        print(key, row[key])


song_id 31477685
year 2019
month 01월
week 12.31~01.06
rank 1

song_id 31532643
year 2019
month 01월
week 12.31~01.06
rank 2

song_id 31406357
year 2019
month 01월
week 12.31~01.06
rank 3

song_id 31417871
year 2019
month 01월
week 12.31~01.06
rank 4

song_id 31448480
year 2019
month 01월
week 12.31~01.06
rank 5

song_id 31506637
year 2019
month 01월
week 12.31~01.06
rank 6

song_id 31532438
year 2019
month 01월
week 12.31~01.06
rank 7


In [13]:
df = pd.DataFrame(rows, columns=rows[0].keys())

,song_id,year,month,week,rank
0,31477685,2019,01월,12.31~01.06,1
1,31532643,2019,01월,12.31~01.06,2
2,31406357,2019,01월,12.31~01.06,3
3,31417871,2019,01월,12.31~01.06,4
4,31448480,2019,01월,12.31~01.06,5
5,31506637,2019,01월,12.31~01.06,6
6,31532438,2019,01월,12.31~01.06,7


In [ ]:
df.to_csv('melon_chart.csv', index = False)